# Learning to Rank or Machine Learned Ranking (MLR)

Download the dataset from https://www.microsoft.com/en-us/research/project/mslr/. We are using the smaller version of the dataset MSLR-WEB10K. The dataset is divided into 5 folds. For this excercise we are using only Fold1

In [1]:
%matplotlib inline

from time import time

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_svmlight_file
from sklearn.metrics import r2_score

In [2]:
dataset_path = "./mslr-web10k/Fold1/"

In [3]:
# You need to pass the query_id=True, otherwise you wont get the query ids
X_train, y_train, train_query_ids = load_svmlight_file(f"{dataset_path}train.txt", query_id=True)

In [4]:
X_test, y_test, test_query_ids = load_svmlight_file(f"{dataset_path}test.txt", query_id=True)

In [5]:
print(X_train.shape, y_train.shape, train_query_ids.shape)
print(X_test.shape, y_test.shape, test_query_ids.shape)

(723412, 136) (723412,) (723412,)
(241521, 136) (241521,) (241521,)


In [6]:
# Reference https://github.com/ogrisel/notebooks/blob/master/Learning%20to%20Rank.ipynb
def subsample(X, y, qid, size, seed=None):
    rng = np.random.RandomState(seed)
    unique_qid = np.unique(qid)
    qid_mask = rng.permutation(len(unique_qid))[:size]
    subset_mask = np.in1d(qid, unique_qid[qid_mask])
    return X[subset_mask], y[subset_mask], qid[subset_mask]

X_train_small, y_train_small, qid_train_small = subsample(
    X_train, y_train, train_query_ids, 500, seed=0)

In [7]:
print(X_train_small.shape, y_train_small.shape, qid_train_small.shape, len(np.unique(qid_train_small)))

(62244, 136) (62244,) (62244,) 500


In [8]:
# Reference https://github.com/ogrisel/notebooks/blob/master/Learning%20to%20Rank.ipynb
def dcg(relevances, rank=10):
    relevances = np.asarray(relevances)[:rank]
    n_relevances = len(relevances)
    if n_relevances == 0:
        return 0

    discounts = np.log2(np.arange(n_relevances) + 2)
    return np.sum(relevances/discounts)

def ndcg(relevances, rank=10):
    best_dcg = dcg(sorted(relevances, reverse=True), rank)
    if best_dcg == 0:
        return 0
    
    return dcg(relevances, rank) / best_dcg

In [9]:
print(dcg([0, 0, 2, 2], 3))
print(dcg([2, 2, 2, 2], 3))
print(dcg([5, 5, 5, 2], 3))
print(dcg([5, 4, 2, 2], 3))
print(dcg([2, 4, 5, 2], 3))

1.0
4.2618595071429155
10.654648767857287
8.523719014285831
7.02371901428583


In [10]:
print(ndcg([0, 0, 2, 2], 3))
print(ndcg([2, 2, 2, 2], 3))
print(ndcg([5, 5, 5, 2], 3))
print(ndcg([5, 4, 2, 2], 3))
print(ndcg([2, 4, 5, 2], 3))

0.3065735963827292
1.0
1.0
1.0
0.8240204777414663


**Compare dcg and ndcg. Observe how the normalization affects ndcg. It only cares about the right order. If the order is right then the actual score of the documents is not considered.**

In [11]:
# Reference https://github.com/ogrisel/notebooks/blob/master/Learning%20to%20Rank.ipynb
def mean_ndcg(y_true, y_pred, query_ids, rank=10):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    query_ids = np.asarray(query_ids)
    
    ndcg_scores = []
    previous_qid = 0
    previous_loc = 0
    for loc, qid in enumerate(query_ids):
        if previous_qid != qid:
            chunk = slice(previous_loc, loc)
            ranked_relevances = y_true[chunk][np.argsort(y_pred[chunk])[::-1]]
            ndcg_scores.append(ndcg(ranked_relevances))
            previous_loc = loc
        previous_qid = qid
        
    chunk = slice(previous_loc, loc+1)
    ranked_relevances = y_true[chunk][np.argsort(y_pred[chunk])[::-1]]
    ndcg_scores.append(ndcg(ranked_relevances))
    return np.mean(ndcg_scores)

In [13]:
mean_ndcg([4, 3, 1, 4, 3], [4, 0, 1, 4, 2], [0, 0, 0, 2, 2], rank=10)

0.9795191506818377

In [12]:
def print_evaluation(model, X, y, qid):
    tic = time()
    y_predicted = model.predict(X)
    prediction_time = time() - tic
    print("Prediction time: {:.3f}s".format(prediction_time))
    print("NDCG@5 score: {:.3f}".format(
    mean_ndcg(y, y_predicted, qid, rank=5)))
    print("NDCG@10 score: {:.3f}".format(
    mean_ndcg(y, y_predicted, qid, rank=10)))
    print("NDCG score: {:.3f}".format(
    mean_ndcg(y, y_predicted, qid, rank=None)))
    print("R2 score: {:.3f}".format(r2_score(y, y_predicted)))

In [16]:
%time
from sklearn.linear_model import LinearRegression

lr = LinearRegression().fit(X_train, y_train)

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 13.4 µs


In [21]:
%time y_test_lr = lr.predict(X_test)
print_evaluation(lr, X_test, y_test, test_query_ids)

CPU times: user 78.2 ms, sys: 1.39 ms, total: 79.6 ms
Wall time: 77.8 ms
Prediction time: 0.067s
NDCG@5 score: 0.435
NDCG@10 score: 0.435
NDCG score: 0.435
R2 score: 0.111


In [22]:
%%time 

from sklearn import svm
clf = svm.SVC(verbose=True)
clf.fit(X_train_small, y_train_small)

/home/sairahul/.local/share/virtualenvs/fastai_v1-1ZccQiGW/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[LibSVM]CPU times: user 2h 3min 40s, sys: 24.8 ms, total: 2h 3min 40s
Wall time: 2h 3min 40s


In [24]:
%%time
print_evaluation(clf, X_test, y_test, test_query_ids)

Prediction time: 3961.430s
NDCG@5 score: 0.263
NDCG@10 score: 0.263
NDCG score: 0.263
R2 score: -0.663
CPU times: user 1h 6min 1s, sys: 139 ms, total: 1h 6min 2s
Wall time: 1h 6min 1s


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.base import RegressorMixin
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone

In [16]:
def proba_to_relevance(probas):
    """MCRank-like reduction of classification proba to DCG predictions"""
    rel = np.zeros(probas.shape[0], dtype=np.float32)
    for i in range(probas.shape[1]):
        rel += i * probas[:, i]
    return rel
        
        
class ClassificationRanker(RegressorMixin):
    
    def __init__(self, base_estimator=None):
        self.base_estimator = base_estimator
        
    def fit(self, X, y):
        self.estimator_ = clone(self.base_estimator)
        self.scaler_ = StandardScaler(with_mean=False)
        X = self.scaler_.fit_transform(X)
        self.estimator_.fit(X, y)
        
    def predict(self, X):
        X_scaled = self.scaler_.transform(X)
        probas = self.estimator_.predict_proba(X_scaled)
        return proba_to_relevance(probas)


In [19]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=100, random_state=1)
gbr = ClassificationRanker(gbc)
gbr.fit(X_train, y_train)

In [29]:
print_evaluation(gbr, X_test, y_test, test_query_ids)


Prediction time: 4.002s
NDCG@5 score: 0.522
NDCG@10 score: 0.522
NDCG score: 0.522
R2 score: 0.172


In [42]:
print(len(train_query_ids))
print(X_train.shape)

723412
(723412, 136)


In [16]:
import xgboost as xgb
from xgboost import DMatrix
import itertools

In [44]:
%%time 

params = {'objective': 'rank:pairwise', 'learning_rate': 0.1,
          'gamma': 0.5, 'min_child_weight': 0.1,
          'max_depth': 6, 'n_estimators': 100}

train_groups = [len(list(values)) for _, values in itertools.groupby(train_query_ids)]

#train_dmatrix = DMatrix(X_train, y_train)
#train_dmatrix.set_group(train_groups)
#xgb_model = xgb.train(params, train_dmatrix)

xgb_model = xgb.sklearn.XGBRanker(**params)
xgb_model.fit(X_train, y_train, train_groups)

CPU times: user 15min 40s, sys: 1.64 s, total: 15min 42s
Wall time: 1min 37s


In [46]:
xgb_model.objective

'rank:pairwise'

In [45]:
print_evaluation(xgb_model, X_test, y_test, test_query_ids)


Prediction time: 6.049s
NDCG@5 score: 0.523
NDCG@10 score: 0.523
NDCG score: 0.523
R2 score: -0.093


In [18]:
xgb_y = xgb_model.predict(X_test)

In [20]:
print(xgb_y[:50])
print(y_test[:50])
print(len(xgb_y))
print(len(y_test))

[0.48516673 0.43478358 0.4598418  0.5314547  0.33852553 0.53118646
 0.4732565  0.46417266 0.5569814  0.50925577 0.54099464 0.43478358
 0.55198264 0.66855407 0.5062158  0.54749286 0.5436234  0.5272356
 0.5314547  0.44697112 0.33852553 0.55436903 0.53807515 0.5272356
 0.49297768 0.4897708  0.38902566 0.53618056 0.54099464 0.54749286
 0.51382136 0.523038   0.43478358 0.45770597 0.60123175 0.50147104
 0.5272356  0.5526739  0.63090694 0.32597935 0.523038   0.4732565
 0.4598418  0.44697112 0.5096137  0.44697112 0.43478358 0.60123175
 0.4897708  0.43478358]
[2. 1. 3. 1. 0. 0. 1. 0. 0. 2. 1. 1. 2. 2. 1. 2. 1. 2. 0. 1. 0. 1. 0. 0.
 1. 0. 0. 2. 2. 2. 1. 1. 0. 0. 1. 0. 0. 1. 3. 0. 1. 1. 1. 0. 1. 1. 1. 0.
 2. 0.]
241521
241521


In [32]:
xgb_y_margin = xgb_model.predict(X_test, output_margin=True)

In [36]:
print(xgb_y_margin[:10])
print(xgb_y[:10])

[0.48516673 0.43478358 0.4598418  0.5314547  0.33852553 0.53118646
 0.4732565  0.46417266 0.5569814  0.50925577]
[0.48516673 0.43478358 0.4598418  0.5314547  0.33852553 0.53118646
 0.4732565  0.46417266 0.5569814  0.50925577]


In [40]:
test_groups = [len(list(values)) for _, values in itertools.groupby(test_query_ids)]

test_dmatrix = DMatrix(X_test)
test_dmatrix.set_group(test_groups)

rank_values = xgb_model.get_booster().predict(test_dmatrix, pred_leaf=True)

In [41]:
rank_values

(241521, 4)

In [29]:
rank_values.shape

(241521,)

# References

* https://github.com/ogrisel/notebooks/blob/master/Learning%20to%20Rank.ipynb